In [ ]:
!pip install -q langchain transformers accelerate bitsandbytes langchain ctransformers sentence-transformers faiss-cpu

Load the libraries

In [ ]:
import torch

from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline

import sys
import textwrap

from langchain.chains import LLMChain
from langchain import HuggingFacePipeline
from langchain import PromptTemplate
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain